# SQL Refresher 1 b

## imports

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

## my_select_query_pandas() - function to run a select query and return rows in a Pandas dataframe

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

## Connect to the Postgres database

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

## Create a cursor for the connection

In [4]:
cursor = connection.cursor()

# Lab: SQL - Set Operations

## Create a table from the results of a query 

In [5]:
connection.rollback()

query = """

drop table if exists temp_zip_1;

drop table if exists temp_zip_2;

create table temp_zip_1
as
select zip, city, state, population
from zip_codes
where state = 'CA' and (city = 'Berkeley' or city = 'Los Angeles')
;

create table temp_zip_2
as
select zip, city, state, population
from zip_codes
where state = 'CA' and (city = 'Los Angeles')
;

"""

cursor.execute(query)

connection.commit()

## Verify the first table we just created

In [9]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from temp_zip_1
order by state, city, zip

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state,population
0,94702,Berkeley,CA,17092
1,94703,Berkeley,CA,21937
2,94704,Berkeley,CA,29190
3,94705,Berkeley,CA,13365
4,94707,Berkeley,CA,11916
...,...,...,...,...
66,90067,Los Angeles,CA,2314
67,90068,Los Angeles,CA,20982
68,90073,Los Angeles,CA,916
69,90077,Los Angeles,CA,8993


## Verify the second table we just created

In [10]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from temp_zip_2
order by state, city, zip

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state,population
0,90001,Los Angeles,CA,58975
1,90002,Los Angeles,CA,53111
2,90003,Los Angeles,CA,72741
3,90004,Los Angeles,CA,61586
4,90005,Los Angeles,CA,39479
...,...,...,...,...
57,90067,Los Angeles,CA,2314
58,90068,Los Angeles,CA,20982
59,90073,Los Angeles,CA,916
60,90077,Los Angeles,CA,8993


## union combines rows from two queries, removing duplicates

In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """

(select *
from temp_zip_1)
union
(select *
from temp_zip_2)
order by state, city, zip

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state,population
0,94702,Berkeley,CA,17092
1,94703,Berkeley,CA,21937
2,94704,Berkeley,CA,29190
3,94705,Berkeley,CA,13365
4,94707,Berkeley,CA,11916
...,...,...,...,...
66,90067,Los Angeles,CA,2314
67,90068,Los Angeles,CA,20982
68,90073,Los Angeles,CA,916
69,90077,Los Angeles,CA,8993


## "union all" combines rows from two queries, keeps duplicates

In [11]:
rollback_before_flag = True
rollback_after_flag = True

query = """

(select *
from temp_zip_1)
union all
(select *
from temp_zip_2)
order by state, city, zip

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state,population
0,94702,Berkeley,CA,17092
1,94703,Berkeley,CA,21937
2,94704,Berkeley,CA,29190
3,94705,Berkeley,CA,13365
4,94707,Berkeley,CA,11916
...,...,...,...,...
128,90073,Los Angeles,CA,916
129,90077,Los Angeles,CA,8993
130,90077,Los Angeles,CA,8993
131,90089,Los Angeles,CA,3862


## intersect returns rows that are in both queries

In [12]:
rollback_before_flag = True
rollback_after_flag = True

query = """

(select *
from temp_zip_1)
intersect
(select *
from temp_zip_2)
order by state, city, zip

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state,population
0,90001,Los Angeles,CA,58975
1,90002,Los Angeles,CA,53111
2,90003,Los Angeles,CA,72741
3,90004,Los Angeles,CA,61586
4,90005,Los Angeles,CA,39479
...,...,...,...,...
57,90067,Los Angeles,CA,2314
58,90068,Los Angeles,CA,20982
59,90073,Los Angeles,CA,916
60,90077,Los Angeles,CA,8993


## except (minus) returns rows in the first query that are not in the second query

In [13]:
rollback_before_flag = True
rollback_after_flag = True

# Note: Most SQL databases support minus.  
# Postgres calls minus except. 
# Set operations tend to be somewhat vendor dependant

query = """

(select *
from temp_zip_1)
except
(select *
from temp_zip_2)
order by state, city, zip

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,city,state,population
0,94702,Berkeley,CA,17092
1,94703,Berkeley,CA,21937
2,94704,Berkeley,CA,29190
3,94705,Berkeley,CA,13365
4,94707,Berkeley,CA,11916
5,94708,Berkeley,CA,11455
6,94709,Berkeley,CA,11740
7,94710,Berkeley,CA,7461
8,94720,Berkeley,CA,2971


## drop our two tables

In [14]:
connection.rollback()

query = """

drop table if exists temp_zip_1;

drop table if exists temp_zip_2;

"""

cursor.execute(query)

connection.commit()

In [23]:
connection.rollback()

query = """

drop table if exists temp_states_1;

drop table if exists temp_states_2;

create table temp_states_1
as
select state_name
from states
where (state_name like 'C%') or (state_name like 'T%')
;

create table temp_states_2
as
select state_name
from states
where state_name like 'T%'
;

"""

cursor.execute(query)

connection.commit()

In [24]:
rollback_before_flag = True
rollback_after_flag = True


query = """

select *
from temp_states_2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,state_name
0,Tennessee
1,Texas


## You try it - using the states table, create temp_states_1 that contains state names that start with a C or a T, create temp_states_2 that contains state names that start with a T, demonstrate set operations, drop the temp tables when you are done

# Lab: SQL - Join Operations

## Joins combine columns from two or more tables; link the primary key in the parent table to the foreign key in the child table; this is an inner join where parent rows without child rows are not included; here we join the stores table to the sales table

In [29]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select stores.city, sum(sales.total_amount) as total_sales
from stores
     join sales
         on stores.store_id = sales.store_id
group by stores.city
order by total_sales desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,city,total_sales
0,Berkeley,25041060
1,Seattle,22024512
2,Dallas,19408260
3,Miami,17692404
4,Nashville,14573172


## Join stores to sales to line_items

In [36]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select s.city, l.product_id, sum(quantity) as total_quantity
from stores as s 
     join sales as sa 
         on s.store_id = sa.store_id
     join line_items as l
         on sa.store_id = l.store_id and sa.sale_id = l.sale_id
group by s.city, l.product_id
order by 1, 3 desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,city,product_id,total_quantity
0,Berkeley,1,464274
1,Berkeley,4,405637
2,Berkeley,6,346508
3,Berkeley,2,290858
4,Berkeley,8,232038
5,Berkeley,7,174252
6,Berkeley,3,115469
7,Berkeley,5,57719
8,Dallas,1,359615
9,Dallas,4,314383


## Join the stores to sales to line_items to products

In [40]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select s.city as store, s.street, p.description as product, sum(quantity) as total_quantity
from stores as s 
     join sales as sa 
         on s.store_id = sa.store_id
     join line_items as l
         on sa.store_id = l.store_id and sa.sale_id = l.sale_id
     join products as p
         on l.product_id = p.product_id
group by s.city, s.street, p.description
order by 1, 3 desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,store,street,product,total_quantity
0,Berkeley,3000 Telegraph Ave,Tilapia Piccata,174252
1,Berkeley,3000 Telegraph Ave,Teriyaki Chicken,290858
2,Berkeley,3000 Telegraph Ave,Spinach Orzo,115469
3,Berkeley,3000 Telegraph Ave,Pistachio Salmon,464274
4,Berkeley,3000 Telegraph Ave,Eggplant Lasagna,405637
5,Berkeley,3000 Telegraph Ave,Curry Chicken,346508
6,Berkeley,3000 Telegraph Ave,Chicken Salad,57719
7,Berkeley,3000 Telegraph Ave,Brocolli Stir Fry,232038
8,Dallas,2510 McKinney Ave,Tilapia Piccata,134451
9,Dallas,2510 McKinney Ave,Teriyaki Chicken,224932


## Join customers to sales to line_items to products

In [42]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select cu.first_name || ' ' || cu.last_name as customer_name,
       p.description as product, 
       sum(quantity) as total_quantity
from customers as cu
     join sales as sa 
         on cu.customer_id = sa.customer_id
     join line_items as l
         on sa.store_id = l.store_id and sa.sale_id = l.sale_id
     join products as p
         on l.product_id = p.product_id
group by customer_name, product
order by 1, 3 desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,customer_name,product,total_quantity
0,Aaren Cossons,Pistachio Salmon,107
1,Aaren Cossons,Eggplant Lasagna,99
2,Aaren Cossons,Curry Chicken,92
3,Aaren Cossons,Teriyaki Chicken,68
4,Aaren Cossons,Brocolli Stir Fry,49
...,...,...,...
248152,Zuzana Luckham,Teriyaki Chicken,48
248153,Zuzana Luckham,Brocolli Stir Fry,39
248154,Zuzana Luckham,Tilapia Piccata,32
248155,Zuzana Luckham,Spinach Orzo,20


## There can be customers who have signed up and not bought anything yet; an inner join will not return them

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select cu.first_name || ' ' || cu.last_name as customer_name,
       sa.sale_date
from customers as cu
     join sales as sa 
         on cu.customer_id = sa.customer_id
where cu.customer_id in (24521, 9318)
order by 1, 2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## left outer join will include parent rows without child rows;  this will include customers who have not bought anything yet

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select cu.first_name || ' ' || cu.last_name as customer_name,
       sa.sale_date
from customers as cu
     left outer join sales as sa 
         on cu.customer_id = sa.customer_id
where cu.customer_id in (24521, 9318)
order by 1, 2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## left outer join must be used on all subsequent joins

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select s.city as store, 
       cu.first_name || ' ' || cu.last_name as customer_name,
       p.description as product, 
       sum(quantity) as total_quantity
from stores as s 
     join customers as cu
         on s.store_id = cu.closest_store_id
     left outer join sales as sa 
         on s.store_id = sa.store_id and cu.customer_id = sa.customer_id
     left outer join line_items as l
         on sa.store_id = l.store_id and sa.sale_id = l.sale_id
     left outer join products as p
         on l.product_id = p.product_id
where cu.customer_id in (24521, 9318)
group by store, customer_name, product
order by 1, 3 desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## You try it - For each store, for each day of the week, for each product, list the total number sold.  

Hints:  

extract(dow from sa.sale_date) will give you the day of week as an integer from 0 to 6 with 0 = Sunday.   You will find this helpful for ordering.

to_char(sa.sale_date, 'Day') will give you the day of week in the form "Sunday", "Monday", etc. You will find this more user friendly than the numeric dow.



# Lab: SQL - Type 1 Subqueries

## type 1 subqueries have no linkage to the outer query;  they can be pulled out and run standalone; can be used in a where clause

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select cu.last_name, cu.first_name, cu.customer_id
from customers as cu
where cu.customer_id not in (select distinct customer_id from sales)
order by 1, 2, 3
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## Type 1 subqueries can also be used in place of table names in from clauses;  the comma is a cross product join (also called Cartesian Product);  match all rows of one table with all rows of another table; good for matching all rows to a total as shown

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.product, 
       a.total_sales_for_product, 
       round((a.total_sales_for_product / b.total_sales) * 100, 1) as percentage_of_total_sales 
from 

    (
     select p.description as product, 
            sum(l.quantity) * 12 as total_sales_for_product
     from sales sa
          join line_items l
              on sa.store_id = l.store_id and sa.sale_id = l.sale_id
          join products p
              on l.product_id = p.product_id
      group by product
    ) as a,
    
    (
     select sum(sa.total_amount) as total_sales 
     from sales sa
    ) as b

order by 3 desc
;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## with clause is an alternative syntax;  most people find it much easier

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

with a as (
            select p.description as product, 
                sum(l.quantity) * 12 as total_sales_for_product
            from sales as sa
            join line_items as l
                on sa.store_id = l.store_id and sa.sale_id = l.sale_id
            join products as p
                on l.product_id = p.product_id
            group by product
          )
          ,
     b as (
            select sum(sa.total_amount) as total_sales 
            from sales as sa     
          )
select a.product, 
       a.total_sales_for_product, 
       round((a.total_sales_for_product / b.total_sales) * 100, 1) as percentage_of_total_sales 
from a, b
order by 3 desc
;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## Views are basically a "with" clause that is permanent and can be shared by all queries; view do not use storage - they are not the same as a "create table as select" that we saw earlier

In [ ]:
connection.rollback()

query = """

drop view if exists v_total_sales;

create or replace view v_total_sales
as
select sum(sa.total_amount) as total_sales 
from sales as sa
;

"""

cursor.execute(query)

connection.commit()

## Views can be used just like tables in queries

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from v_total_sales
;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## View to join stores to sales to customers; also creates several useful derived columns

In [ ]:
connection.rollback()

query = """

drop view if exists v_join_s_sa_cu;

create view v_join_s_sa_cu
as
select s.store_id,
       s.street as store_street,
       s.city as store_city,
       s.state as store_state,
       s.zip as store_zip,
       s.latitude as store_latitude,
       s.longitude as store_longitude,
       sa.sale_id,
       sa.customer_id,
       sa.sale_date,
       extract(dow from sa.sale_date) as dow,
       to_char(sa.sale_date, 'Day') as day_of_week,
       extract(month from sa.sale_date) as month_number,
       to_char(sa.sale_date, 'Month') as month_name,
       sa.total_amount,
       cu.first_name,
       cu.last_name,
       (cu.first_name || ' ' || cu.last_name) as full_name,
       (cu.last_name || ', ' || cu.first_name) as last_name_first,
       cu.street as customer_street,
       cu.city as customer_city,
       cu.state as customer_state,
       cu.zip as customer_zip,
       cu.distance
from stores s
     join sales as sa
        on s.store_id = sa.store_id
     join customers as cu
        on sa.customer_id = cu.customer_id
;

"""

cursor.execute(query)

connection.commit()

## This join is huge! Remeber to use a limit, an aggregation, where clause, etc. to limit the return!

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from v_join_s_sa_cu
limit 5
;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## View to join stores to sales to customer to line_items to products

In [ ]:
connection.rollback()

query = """

drop view if exists v_join_s_sa_cu_l_p;

create view v_join_s_sa_cu_l_p
as
select s.store_id,
       s.street as store_street,
       s.city as store_city,
       s.state as store_state,
       s.zip as store_zip,
       s.latitude as store_latitude,
       s.longitude as store_longitude,
       sa.sale_id,
       sa.customer_id,
       sa.sale_date,
       extract(dow from sa.sale_date) as dow,
       to_char(sa.sale_date, 'Day') as day_of_week,
       extract(month from sa.sale_date) as month_number,
       to_char(sa.sale_date, 'Month') as month_name,
       sa.total_amount,
       cu.first_name,
       cu.last_name,
       (cu.first_name || ' ' || cu.last_name) as full_name,
       (cu.last_name || ', ' || cu.first_name) as last_name_first,
       cu.street as customer_street,
       cu.city as customer_city,
       cu.state as customer_state,
       cu.zip as customer_zip,
       cu.distance,
       l.line_item_id,
       l.product_id,
       l.quantity,
       (l.quantity * 12) as line_item_amount,
       p.description as product
from stores s
     join sales as sa
        on s.store_id = sa.store_id
     join customers as cu
        on sa.customer_id = cu.customer_id
     join line_items as l
        on sa.store_id = l.store_id and sa.sale_id = l.store_id
     join products as p
        on l.product_id = p.product_id
;

"""

cursor.execute(query)

connection.commit()

## An even bigger join - be careful and use a limit, aggregation, where, etc.

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from v_join_s_sa_cu_l_p
limit 5
;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## You try it - Create a view called v_store_sales_by_day_of_week that finds each store's sales by day of week. Display the store id, city, day of week in numeric for sorting purposes, day of week in string for display purposes, and total_sales. Test the view by selecting from it.  You can drop the view when you are done with it if you want.

# Lab: SQL - Type 2 Subqueries

## Type 2 subqueries have a linkage to the outer query;   cannot be pulled out and run standalone; notorously slow and high memory usage - be careful! in the where clause, zip in the subquery is linked to cu.zip in the outer query

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select cu.customer_id, 
       cu.first_name, 
       cu.last_name,
       sum(sa.total_amount) as total_sales
from customers cu
     join sales as sa
         on cu.customer_id = sa.customer_id
where (select population from zip_codes where zip = cu.zip) > 50000
group by cu.customer_id, cu.first_name, cu.last_name
;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## Type 2 subqueries can also go in the having clause on aggregations

In [ ]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select cu.customer_id, 
       cu.first_name, 
       cu.last_name,
       sum(sa.total_amount) as total_sales
from customers cu
     join sales as sa
         on cu.customer_id = sa.customer_id
where cu.zip = '94720'
group by cu.customer_id, cu.first_name, cu.last_name
having sum(sa.total_amount) >
        (select avg(sa2.total_amount)
         from stores as s2
              join sales as sa2
                  on s2.store_id = sa2.store_id
         where s2.store_id = cu.closest_store_id)
;


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

## You try it - using a type 2 subquery: find customers in Alameda, CA whose total sales is greater than the average sales for their zip code.  Display the customer_id, first_name, last_name, and total_sales. (this may take a while to run)